# Extracting report related information from .docx files

This notebook is to test the functionalities without having to run the whole script repeatedly.

It automatically extracts the contents of a Word document, according to the given format, saves the extracted information into dictionary variables, which are pickled and stored locally.

In [1]:
from docx.text.paragraph import Paragraph
from docx.document import Document
from docx.table import _Cell, Table
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
import docx
from openai import OpenAI
import pickle
import re

iter_bock_items() is the function used to iterate over paragraphs and tables in a docx document

In [2]:
def iter_block_items(parent):
    if isinstance(parent, Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            #yield Paragraph(child, parent)
            current_p=Paragraph(child,parent)
            yield current_p
        elif isinstance(child, CT_Tbl):
            table = Table(child, parent)
            for row in table.rows:
                for cell in row.cells:
                    yield from iter_block_items(cell)

iter_block_items_n() iterates over paragraphs and tables and recognises rows and cells

In [3]:
rownr=0
cellnr=0
def iter_block_items_n(parent):
    global rownr, cellnr
    if isinstance(parent, Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            #yield Paragraph(child, parent)
            current_p=Paragraph(child,parent)
            #print(rownr,cellnr)
            if not isinstance(current_p._parent,_Cell):
                rownr=0
                cellnr=0
            yield current_p
        elif isinstance(child, CT_Tbl):
            table = Table(child, parent)
            rownr=0
            cellnr=0
            for row in table.rows:
                cellnr=0
                rownr +=1
                for cell in row.cells:
                    cellnr+=1
                    yield from iter_block_items_n(cell)

Read the desired Word document
(Can't run if the document is open)

In [151]:
doc = docx.Document('Projectreports/23-1111_JT Projektreport.docx')

Initialise Dictionaries that will store the information from the Word document (RN: untill chapter 5: Macht und Interessen)

Infos that do not belong to any Chapter:

In [5]:
#Erste Tabelle Titelseite
D_Titelseite={'Projekt':'',
              'Teilnehmer':'',
              'Firma':'',
              'Adresse':'',
              'Telefon':'',
              'E-Mail':'',
              'Kurs':''}

#Änderungshistorie Tabelle
D_Änderungshistorie={} #Further entries are added later on 

#I name this entries chapter 0
D_Chapt_0={'Angaben zum Projekt':'',
           'Eigene Position':''}




Infos from chapt 1. Strategie 4.5.1.

In [6]:
D_Strategie={'Beschreibung des Business Case zum Projekt':'',
                'Nennung der kritischen Erfolgsfaktoren des Projekts':''}

Infos from chapt 2. Governance, Strukturen und Prozesse 04.03.02

In [7]:
D_Governance_Strukturen_Prozesse={'Begründung, warum es sich bei dem Vorhaben um ein Projekt handelt':'',
                                  'Nennung der Projektart des Projektes und Begründung der Einordnung':'',
                                  'Nennung und Begründung der Klassifizierung des Projekts aus Sicht der Organisation':'',
                                  'Nennung der Strukturen der Organisation':''}

Infos from chapt 3. Anforderungen und Ziele 4.5.2.

In [132]:
D_Steckbrief={'Projektbezeichnung':'', 
              'Projektnummer':'',
              'Auftraggeber':'',
              'Projektleitung, Verantwortung':'',
              'Leistung':None,
              'Termine':None,
              'Kosten':None,
              'Projektorganisationsform':'',
              'Stabs-PO':None,
              'Matrix-PO':None,
              'Reine_PO':None,
              'Oberziel':'',
              'Projektinhalt':'',
              'Projektbeteiligte':'',
              'Projektumfeld':'',
              'Starttermin':'',
              'Endetermin': '',
              'Dauer': '',
              'Zwischentermine':'',
              'Aufwand-gesamt':'',
              'Aufwand-intern':'',
              'Aufwand-extern':'',
              'Personalkosten-gesamt':'',
              'Personalkosten-intern':'',
              'Personalkosten-extern':'',
              'Sachkosten-gesamt':'',
              'Sachkosten-intern':'',
              'Sachkosten-extern':'',
              'Investitionen-gesamt':'',
              'Investitionen-intern':'',
              'Investitionen-extern':'',
              'Budget-gesamt':'',
              'Budget-intern':'',
              'Budget-extern':'',
              'vrsl. Behinderungen/Risiken/Störungen':'',
              'Kunde':'',
              'Abnahmekriterien':''}

D_Oberziel={'Bezeichnung':'', 'Formulierung':'', 'Messkriterium':''}

#Ergebnisziele:
D_Finanzziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}           #Further entries are added later on 
D_Leistungsziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}        #Further entries are added later on 
D_Qualitätsziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}        #Further entries are added later on 
D_SozialeZiele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}          #Further entries are added later on 

#Vorgehensziele:
D_Terminziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}           #Further entries are added later on 
D_Kostenziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}           #Further entries are added later on 
D_Aufwandsziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}         #Further entries are added later on 
D_Projektrahmenziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}    #Further entries are added later on 

#Nicht-Ziele:
D_NichtZiele={}                                  #Further entries are added later on 

#Zielkonflikte
D_Zielkonflikte={'1-Ziel_a':'',
                 '1-Ziel_b':'',
                 '1-Konfliktart':'',
                 '1-Priorität':'',
                 '1-Erklärung':'',
                 '1-Massnahmen':''} #Further entries are added later on 

Infos from chapt 4. Stakeholder 4.5.12.

In [9]:
D_Projektumfeld={'sozial-intern':'',
                 'sozial-extern':'',
                 'sachlich-intern':'',
                 'sachlich-extern':''}

D_Schnittstellen_Stakeholder_Umfeld={} #Further entries may be added later on 

D_Stakeholderinteressen={'1-Stakeholder':'',
                         '1-Konfliktpotentzial':'',
                         '1-Einfluss/Macht':'',
                         '1-Erwartungen/Befrüchtungen':'',
                         '1-Strategie':'',
                         '1-Steuerung/Massnahmen':''} #Further entries are added later on 

Infos in chapt. 5. Macht und Interessen 04.03.04

In [10]:
#Maybe fuse into one dictionary
D_Stakeholderbewertung={} #Further entries are added later on 

D_Machtpromotoren={'Machtpromotoren':''}

Prints all text blocks

In [11]:
past_block=''
for block in iter_block_items(doc):
    current_block=block.text
    # ignore repeated cells, #ignore paragraphs starting with ignore_list
    if current_block != past_block: #and not current_block.startswith(ignore_list[0]) and not current_block.startswith(ignore_list[1]) and not current_block.startswith(ignore_list[2]) and not current_block.startswith(ignore_list[3]) and not current_block.startswith(ignore_list[4])
        print(block.text)
        #print(block.style)
    past_block=current_block

Report Level D

nach Z01D_Leitfaden / 02 vom 20.06.2023

Dieses Dokument basiert auf der Report-Vorlage der Decisio Projekt- und Prozessmanagement GmbH. 

Projekt:
Servicemigration IT-Servicecenter
Teilnehmer:
JT
Firma:
ABC
Adresse:

Telefon:

E-Mail:

Kurs:
23-1111

Änderungshistorie	
Version
Datum
Ersteller
Grund
0.1
26.09.23
JT
Ersterstellung
0.2
26.11.23
JT
Überarbeitung
1.0
tt.mm.jj
Autor
Fertigstellung

Hinweis zur Bearbeitung

Zum Abschluss, kurz vor dem Upload, bitte die Versionsnummer in der Fußzeile anpassen.

Inhaltsverzeichnis
1.	Strategie 4.5.1.	6
1.1.	Beschreibung des Business Case zum Projekt	6
1.2.	Nennung der kritischen Erfolgsfaktoren des Projekts	6
2.	Governance, Strukturen und Prozesse 04.03.02	7
2.1.	Begründung, warum es sich bei dem Vorhaben um ein Projekt handelt	7
2.2.	Nennung der Projektart des Projektes und Begründung der Einordnung	7
2.3.	Nennung und Begründung der Klassifizierung des Projekts aus Sicht der Organisation	7
2.4.	Nennung der Strukturen der Organ

Betriebsvertrag und Abnahmeerklärung liegen vor
Standardisierte Service- & Betriebsprozesse sind eingeführt & die IT-Betriebsverantwortung wurde übernommen.
Vollständige Prozessdokumentation
Betriebsvertrag und Abnahmeerklärung liegen vor
Standardisierte Service- & Betriebsprozesse sind eingeführt & die IT-Betriebsverantwortung wurde übernommen.
Vollständige Prozessdokumentation
Betriebsvertrag und Abnahmeerklärung liegen vor
Standardisierte Service- & Betriebsprozesse sind eingeführt & die IT-Betriebsverantwortung wurde übernommen.
Unterschrift 

Auftraggeber

Auftraggeber

Projektleiter

Projektleiter

Projektleiter

Projektleiter

Projektleiter
Tabelle 1 Steckbrief
Ziele

Nr.
Hierarchie
Ziel-bezeichnung
Ziel-formulierung
Mess-kriterium
Kategorie / Priorität
1
Oberziel
Einführung standardisierter Service- u. Betriebsprozesse inkl. Übernahme IT-Betriebs-verantwortung 
Einführung standardisierter Service- u. Betriebsprozesse werden Aufgaben u. Prozesse der IT-Abteilung der Bank an die 

Function to clear all auxiliary search variables

In [136]:
def clear_search_variables():
    global textvar, past_block, current_block, found_Cell, found_Chapter, found_Table, past_Cell, og_cell
    textvar=""
    past_block=''
    current_block=""
    past_Cell=""
    found_Chapter=False
    found_Cell=False
    found_Table=False
    og_cell=None

## Chapter 0

Saves the infos of the first table in the cover page

In [12]:
search_list=['Projekt:','Teilnehmer:', 'Firma:', 'Adresse:','Telefon:','E-Mail:','Kurs:']
save_D=D_Titelseite
save_list=save_D.keys()
for search_item, save_item in zip(search_list,save_list):
    #print(search_item, save_item)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith(search_item):
            found_Cell=True
            og_cell=current_cell
        # print(current_cell)
        # print(found_Cell)
        # print(block.text)
        if found_Cell:
            if current_cell!=og_cell: 
                break
            textvar+=block.text #took this line into the if condition
            textvar+="\n"
        past_block=block
        past_Cell=current_cell
    textvar=textvar.strip() #removes eventual whitespaces in the begining or end
    D_Titelseite[save_item]=textvar
print(D_Titelseite)

{'Projekt': 'Servicemigration IT-Servicecenter', 'Teilnehmer': 'JT', 'Firma': 'ABC', 'Adresse': '', 'Telefon': '', 'E-Mail': '', 'Kurs': '23-1111'}


Saves Änderungshistorie table

Counts the number of versions (nr of rows - 1) in the table

In [13]:
clear_search_variables()
for block in iter_block_items_n(doc):
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Änderungshistorie') and current_cell==(1,1):
        found_Table=True
    if found_Table and current_cell==(0,0):
        print('Number of versions: ', end='')
        print(past_Cell[0]-1)
        break
    past_Cell=current_cell
    past_block=block

Number of versions: 3


This code repeats itself until it exits the Table we are looking for

In [14]:
clear_search_variables()
D_Änderungshistorie={} #Further entries are added later on 
left_Table=False
flag_cell=(1,4)
while not (found_Table and current_cell==(0,0)):
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Änderungshistorie') and current_cell==(1,1):
            found_Table=True
        if found_Table and current_cell==(0,0):
            left_Table=True
            #break
        if found_Table and past_Cell==flag_cell and current_cell[0]>flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        past_Cell=current_cell
        past_block=block
    textvar=textvar.strip() #removes eventual whitespaces in the begining or end    
    version_nr=textvar
    new_D_key='Version_'+version_nr
    flag_cell=og_cell
    D_Änderungshistorie[new_D_key]={}
    #print(D_Änderungshistorie)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    # print('flag_cell: ', end='')
    # print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Änderungshistorie') and current_cell==(1,1):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block
    textvar=textvar.strip() #removes eventual whitespaces in the begining or end
    version_datum=textvar
    new_D_value=version_datum
    flag_cell=og_cell
    D_Änderungshistorie[new_D_key]['Datum']=new_D_value
    #print(D_Änderungshistorie)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Änderungshistorie') and current_cell==(1,1):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        past_Cell=current_cell
        past_block=block
    textvar=textvar.strip() #removes eventual whitespaces in the begining or end
    version_ersteller=textvar
    new_D_value=version_ersteller
    flag_cell=og_cell
    D_Änderungshistorie[new_D_key]['Ersteller']=new_D_value
    #print(D_Änderungshistorie)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Änderungshistorie') and current_cell==(1,1):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        past_Cell=current_cell
        past_block=block
    
    textvar=textvar.strip() #removes eventual whitespaces in the begining or end
    version_grund=textvar
    new_D_value=version_grund
    flag_cell=og_cell
    D_Änderungshistorie[new_D_key]['Grund']=new_D_value
    #print(D_Änderungshistorie)
print(D_Änderungshistorie)

{'Version_0.1': {'Datum': '26.09.23', 'Ersteller': 'JT', 'Grund': 'Ersterstellung'}, 'Version_0.2': {'Datum': '26.11.23', 'Ersteller': 'JT', 'Grund': 'Überarbeitung'}, 'Version_1.0': {'Datum': 'tt.mm.jj', 'Ersteller': 'Autor', 'Grund': 'Fertigstellung'}}


Looks for paragraphs after 'Angaben zum Projekt' and 'Eigene Position' and saves them into the corresponding dictionaries

In [15]:
clear_search_variables()
for block in iter_block_items(doc):
    current_block=block
    # don't need to ignore repeated text from combined cells for this case
    #Are we in the section we're looking for based on a previous block?
    if past_block!='' and past_block.text.startswith('Angaben zum Projekt'):
        found_Chapter=True
    #Break argument. When are we leaving the section we're lookin for?
    if found_Chapter:
        if block.text.startswith('Eigene Position'):
            break
        textvar+=block.text
        textvar+="\n"
    past_block=current_block
textvar=textvar.strip() #removes eventual whitespaces in the begining or end
D_Chapt_0['Angaben zum Projekt']=textvar
print(D_Chapt_0['Angaben zum Projekt'])

Bei dem Projekt „Servicemigration IT-Servicecenter“ geht es um die Einführung standardisierter Service- und Betriebsprozesse in der IT-Abteilung der Beckedorfer Bank, durch die ABC GmbH.

Die ABC GmbH ist IT-Dienstleister für Banken und bietet unterschiedliche Leistungen rund um die Entwicklung, Produktion und Bereitstellung von Banking-Lösungen an.

Durch die Einführung werden Aufgaben und Prozesse der IT-Abteilung der Bank an die ABC GmbH teils übertragen, neu strukturiert und gegebenenfalls erweitert. Diese Prozesse enthalten unter 
anderem das Auftrags-/Angebots-, Incident-, Asset- und Endgerätemanagement, Kommunikations- und Netzwerkservices.

Auftraggeber ist der Geschäftsführer der Beckedorfer Bank Herr Stephan Bonka, der das Budget in Höhe von 120.485,02 € zur Verfügung stellt. Der Aufwand für die Migration beläuft sich auf 214 PT für eine Dauer von 12 Monaten.

Im Rahmen der Ressourcenplanung hat man ein fachübergreifendes Projektteam für die Migration bereitgestellt. Die Mita

In [16]:
# The text of the chapter will be saved into this variable
clear_search_variables()
for block in iter_block_items(doc):
    current_block=block
    # don't need to ignore repeated text from combined cells for this case
    #Are we in the section we're looking for based on a previous block?
    if past_block!='' and past_block.text.startswith('Eigene Position'):
        found_Chapter=True
    #Break argument. When are we leaving the section we're lookin for?
    if found_Chapter:
        if block.text.startswith('Strategie 4.5.1') or block.style.name=='Heading 2' or block.style.name=='Heading 1':
            break
        textvar+=block.text
        textvar+="\n"
    past_block=current_block
textvar=textvar.strip() #removes eventual whitespaces in the begining or end
D_Chapt_0['Angaben zum Projekt']=textvar
print(D_Chapt_0['Angaben zum Projekt'])

JT ist Projektleiterin im Team Neukundenmigration in der Abteilung Projektservices und hat in Abstimmung mit Ihrem Abteilungsleiter Herr Franz Groitzsch die Projektleitung für das Projekt „Servicemigration IT-Servicecenter“ übernommen.
Sie ist erste Ansprechpartnerin seitens der Fachbereiche der Bank.
Zur Erfüllung dieser Aufgabe übernimmt Sie die Leitung des Projektes und ist dem Auftraggeber gegenüber verantwortlich für die Erreichung der im Auftrag definierten Projektziele hinsichtlich der Termine, des Budgets, Leistungsumfangs und der Qualität. 
Eine rechtzeitige Eskalation bei auftretenden Problemen, die außerhalb Ihres Fachbereichs liegen, verantwortet Sie ebenso. Sie fordert die Ressourcen an und verteilt die Aufgaben innerhalb des Projektteams.
Die Kontrolle und Steuerung aller Arbeitsergebnisse im Projekt zählen zudem zu Ihren Kompeten-zen.


## CHAPTER 1: Strategie 4.5.1. 

Looks for paragraphs in Subchapter 1.1 'Beschreibung des Business Case zum Projekt'  and saves them into a directory

In [17]:
# The text of the chapter will be saved into this variable
clear_search_variables()
for block in iter_block_items(doc):
    current_block=block
    # don't need to ignore repeated text from combined cells for this case
    #Are we in the section we're looking for based on a previous block?
    if past_block!='' and past_block.style.name=='Heading 2' and past_block.text.startswith('Beschreibung des Business Case zum Projekt'):
        found_Chapter=True
    #Break argument. When are we leaving the section we're lookin for?
    if found_Chapter:
        if block.style.name=='Heading 2' or block.style.name=='Heading 1':
            break
        textvar+=block.text
        textvar+="\n"
    #print(block.style.name)
    past_block=current_block
textvar=textvar.strip() #removes eventual whitespaces in the begining or end
D_Strategie['Beschreibung des Business Case zum Projekt']=textvar
print(D_Strategie['Beschreibung des Business Case zum Projekt'])

Das beschriebene Projekt weist einen besonderen Nutzen vor, wenn das fixe Gesamtbudget des Projektes in Höhe von 120.485,02 € nicht überschritten wird und darüber hinaus alle Projektziele innerhalb des ZeABClanes erreicht werden.
Die monatlichen Kosten des Servicecenters der ABC GmbH belaufen sich auf 32.000 €, die gegen-über der monatlichen Kosten des IT Betriebes der Beckedorfer Bank in Höhe von 46.000 €, zu ei-ner Reduzierung von 14.000 € führen. Demnach wird sich eine Amortisation innerhalb von 13 Mo-naten einstellen.
Weitere Nutzen stellen sich durch die Übernahme von Aufgaben einschließlich der Erfüllung von aufsichtsrechtlichen Anforderungen durch die ABC GmbH, die Minimierung von Betriebsrisiken, der Zugang zu externem Spezialistenwissen, Effizienzsteigerungen und Qualitätsverbesserungen und einem flexiblen Kostenmodell dar, welches eine bedarfsgerechte Skalierung ermöglicht.


Looks for paragraphs in Subchapter 1.2 'Nennung der kritischen Erfolgsfaktoren des Projekts' and saves them into a directory

In [18]:
# The text of the chapter will be saved into this variable
clear_search_variables()
for block in iter_block_items(doc):
    current_block=block
    # don't need to ignore repeated text from combined cells for this case
    #Are we in the section we're looking for based on a previous block?
    if past_block!='' and past_block.style.name=='Heading 2' and past_block.text.startswith('Nennung der kritischen Erfolgsfaktoren des Projekts'):
        found_Chapter=True
    #Break argument. When are we leaving the section we're lookin for?
    if found_Chapter:
        if block.style.name=='Heading 2' or block.style.name=='Heading 1':
            break
        textvar+=block.text
        textvar+="\n"
    #print(block.style.name)
    past_block=current_block
textvar=textvar.strip() #removes eventual whitespaces in the begining or end
D_Strategie['Nennung der kritischen Erfolgsfaktoren des Projekts']=textvar
print(D_Strategie['Nennung der kritischen Erfolgsfaktoren des Projekts'])

Als kritische Erfolgsfaktoren konnten für dieses Projekt identifiziert werden:

Damit das Projekt überhaupt starten kann, muss der Migrationsvertrag vor dem offiziellen Start zum 01.09.22 vorliegen. Dieser wird als Projektauftrag gesehen.

Die Vorlage des Betriebsvertrages ist für den Betriebsübergang (Cut Over) notwendig, da dieser sonst offiziell nicht durchgeführt werden darf. Hierzu erfolgt eine frühzeitige Abstimmung mit dem Kunden und Anlauf der offiziellen Prozesse.

Die Projektergebnisse werden in die Linie übergeben, die in das Projekt eingebunden wer-den. Die Projektergebnisse werden standardisiert.

Bestehende Hardware läuft während des Projektes aus dem EOS und muss innerhalb von 2 Monaten getauscht werden. Durch frühzeitige Kontrolle und Sensibilisierung der Beckedorfer Bank erfolgt ein Austausch aller Systeme vor dem Cut Over.

Mitarbeiter der Bank sind im Besitz von Administrationsrechten, die zum Betriebsübergang abgegeben werden müssen, da die Verantwortung ab diesem Z

## Chapter 2: Governance, Strukturen und Prozesse 04.03.02 

OBSERVATION:

IF SOMEONE ADDS AN 'ENTER' INSIDE A PARAGRPH FOR (PROBABLY) AESTHETIC REASONS, THE ALGORITHM WILL IDENTIFY IT AS A NEW PRAGRAPH.

Looks for paragraphs in Subchapter 2.1 'Begründung, warum es sich bei dem Vorhaben um ein Projekt handelt' and saves them into a directory

In [19]:
# The text of the chapter will be saved into this variable
clear_search_variables()
for block in iter_block_items(doc):
    current_block=block
    # don't need to ignore repeated text from combined cells for this case
    #Are we in the section we're looking for based on a previous block?
    if past_block!='' and past_block.style.name=='Heading 2' and past_block.text.startswith('Begründung, warum es sich bei dem Vorhaben um ein Projekt handelt'):
        found_Chapter=True
    #Break argument. When are we leaving the section we're lookin for?
    if found_Chapter:
        if block.style.name=='Heading 2' or block.style.name=='Heading 1':
            break
        textvar+=block.text
        textvar+="\n"
    #print(block.style.name)
    past_block=current_block
textvar=textvar.strip() #removes eventual whitespaces in the begining or end
D_Governance_Strukturen_Prozesse['Begründung, warum es sich bei dem Vorhaben um ein Projekt handelt']=textvar
print(D_Governance_Strukturen_Prozesse['Begründung, warum es sich bei dem Vorhaben um ein Projekt handelt'])

Das beschriebene Projekt „Servicemigration IT-Servicecenter“ hat in dem begrenzten Zeitraum vom 01.09.2022 bis 01.09.2023 stattgefunden und verfügt über ein Budget in Höhe von 120.485,02 € mit 214 PT.

Mit der ‘‘Einführung standardisierter Service- und Betriebsprozesse‘‘ ist das Projektoberziel definiert.

Das Projekt ist mit regulatorischen und terminlichen Risken behaftet, wie z.B. Anforderungen der BaFin an die Bank und einer möglichen Verschiebung des Betriesbübergangs.

Der Auftraggeber dieses Projektes ist der Geschäftsführer Herr Stephan Bonka der Beckedorfer Bank. Das Projekt findet in einer Matrixorganisation statt und setzt sich fachübergreifend aus 
Mitarbeitern aus den Abteilungen SC Projektservices, SC Produktservices (MA Linie), 
SC Auftragsmanagement (MA Linie) und SC Incidentmanagement (MA Linie) zusammen.


Looks for paragraphs in Subchapter 2.2 'Nennung der Projektart des Projektes und Begründung der Einordnung' and saves them into a directory

In [20]:
# The text of the chapter will be saved into this variable
clear_search_variables()
for block in iter_block_items(doc):
    current_block=block
    # don't need to ignore repeated text from combined cells for this case
    #Are we in the section we're looking for based on a previous block?
    if past_block!='' and past_block.style.name=='Heading 2' and past_block.text.startswith('Nennung der Projektart des Projektes und Begründung der Einordnung'):
        found_Chapter=True
    #Break argument. When are we leaving the section we're lookin for?
    if found_Chapter:
        if block.style.name=='Heading 2' or block.style.name=='Heading 1':
            break
        textvar+=block.text
        textvar+="\n"
    #print(block.text)
    past_block=current_block
D_Governance_Strukturen_Prozesse['Nennung der Projektart des Projektes und Begründung der Einordnung']=textvar.strip()
print(D_Governance_Strukturen_Prozesse['Nennung der Projektart des Projektes und Begründung der Einordnung'])

Bei dem Projekt „Servicemigration IT-Servicecenter“ geht es um die Einführung standardisierter Service- und Betriebsprozesse in der IT-Abteilung der Beckedorfer Bank, durch die ABC GmbH.

Durch die Einführung werden Aufgaben und Prozesse der IT-Abteilung der Bank an die ABC GmbH teils übertragen, neu strukturiert und gegebenenfalls erweitert. Es handelt sich deshalb um ein Organisationsprojekt, das bei der ABC als Migrationsprojekt bezeichnet wird.


Looks for paragraphs in Subchapter 2.3 'Nennung und Begründung der Klassifizierung des Projekts aus Sicht der Organisation' and saves them into a directory

In [21]:
# The text of the chapter will be saved into this variable
clear_search_variables()
for block in iter_block_items(doc):
    current_block=block
    # don't need to ignore repeated text from combined cells for this case
    #Are we in the section we're looking for based on a previous block?
    if past_block!='' and past_block.style.name=='Heading 2' and past_block.text.startswith('Nennung und Begründung der Klassifizierung des Projekts aus Sicht der Organisation'):
        found_Chapter=True
    #Break argument. When are we leaving the section we're lookin for?
    if found_Chapter:
        if block.style.name=='Heading 2' or block.style.name=='Heading 1':
            break
        textvar+=block.text
        textvar+="\n"
    #print(block.text)
    past_block=current_block
D_Governance_Strukturen_Prozesse['Nennung und Begründung der Klassifizierung des Projekts aus Sicht der Organisation']=textvar.strip()
print(D_Governance_Strukturen_Prozesse['Nennung und Begründung der Klassifizierung des Projekts aus Sicht der Organisation'])

Das externe und nationale Projekt, welches durch die Beckedorfer Bank in Auftrag gegeben wurde, gilt bei der ABC GmbH mit einem Budget von 120.485,02 € als mittleres Projekt. Die Priorität wird bei diesem Projekt als hoch angesehen und misst diesem aus Sicht der Organisation eine niedrige strategische Bedeutung bei. Eine Gefährdung für die Organisation ist nicht gegeben, 
sollten Projektziele nicht erreicht werden.


Looks for paragraphs in Subchapter 2.4 'Nennung der Strukturen der Organisation' and saves them into a directory

In [22]:
# The text of the chapter will be saved into this variable
clear_search_variables()
for block in iter_block_items(doc):
    current_block=block
    # don't need to ignore repeated text from combined cells for this case
    #Are we in the section we're looking for based on a previous block?
    if past_block!='' and past_block.style.name=='Heading 2' and past_block.text.startswith('Nennung der Strukturen der Organisation'):
        found_Chapter=True
    #Break argument. When are we leaving the section we're lookin for?
    if found_Chapter:
        if block.style.name=='Heading 2' or block.style.name=='Heading 1':
            break
        textvar+=block.text
        textvar+="\n"
    #print(block.text)
    past_block=current_block
D_Governance_Strukturen_Prozesse['Nennung der Strukturen der Organisation']=textvar.strip()
print(D_Governance_Strukturen_Prozesse['Nennung der Strukturen der Organisation'])

Bei dem beschriebenen Projekt „Servicemigration IT-Servicecenter“ geht es um ein Einzelprojekt.

Die Projektleiterin der ABC GmbH koordiniert und steuert das Projekt und das Projektteam in 
Zusammenarbeit mit seiner Stellvertreterin. Sie verantwortet das Projekt gegenüber dem Kunden und innerhalb der ABC GmbH und ist damit Ansprechpartnerin und erste Eskalationsinstanz.

Das Projektoffice unterstützt die Projektleiterin und das Projektteam bei ihren Aufgaben hinsichtlich Organisation, Nachverfolgung von Aufgaben und Initiieren von Abstimmungen. Das Projektteam ist aufgeteilt in Arbeitspaketverantwortliche, welche für ihr jeweiliges Fachthema und die zugehörigen Aktivitäten im Migrationsplan verantwortlich sind.

Zum Lenkungsausschuss gehören der zuständige Geschäftsführer, Abteilungsleiter, Auftraggeber und auf Anforderung der Geschäftsführung auch die Projektleiterin.


## Chapter 3: Anforderungen und Ziele 4.5.2. 

TO DO

3.1 Extracts infos from the Steckbrief table

Extracts all "easy" cells:
* Projektbezeichnung
* Projektnummer
* Auftraggeber (Name, Position)
* Projektleitung
* Projektorganisationsform
* Oberziel
* Projektinhalt (was?)
* Projektbeteiligte (wer ?)
* Projektumfeld
* Zwischentermine (mit Auftraggeber vereinbart)
* Vorauss. Behinderungen /Risiken/ Störungen/Chance
* Kunde
* Abnahmekriterien





In [153]:
#Projektbezeichnung
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Projektbezeichnung:'):
        found_Cell=True
        og_cell=current_cell
    #print(current_cell)
    #print(found_Cell)
    #print(block.text)
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell
D_Steckbrief['Projektbezeichnung']=textvar

###################################################################################################
#Projektnummer
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Projektnummer'):
        found_Cell=True
        og_cell=current_cell
    #print(current_cell)
    #print(found_Cell)
    #print(block.text)
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell
D_Steckbrief['Projektnummer']=textvar

###################################################################################################
#Auftraggeber
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Auftraggeber (Name'):
        found_Cell=True
        og_cell=current_cell
    #print(current_cell)
    #print(found_Cell)
    #print(block.text)
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell
D_Steckbrief['Auftraggeber']=textvar

###################################################################################################
#Projektleitung
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Projektleitung'):
        found_Cell=True
        og_cell=current_cell
    #print(current_cell)
    #print(found_Cell)
    #print(block.text)
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell
D_Steckbrief['Projektleitung']=textvar

###################################################################################################
#Projektorganisationsform
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Projektorganisationsform'):
        found_Cell=True
        og_cell=current_cell
    #print(current_cell)
    #print(found_Cell)
    #print(block.text)
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell
D_Steckbrief['Projektorganisationsform']=textvar

###################################################################################################
#Oberziel
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Oberziel'):
        found_Cell=True
        og_cell=current_cell
    #print(current_cell)
    #print(found_Cell)
    #print(block.text)
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell
D_Steckbrief['Oberziel']=textvar

###################################################################################################
#Projektinhalt
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Projektinhalt'):
        found_Cell=True
        og_cell=current_cell
    #print(current_cell)
    #print(found_Cell)
    #print(block.text)
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell
D_Steckbrief['Projektinhalt']=textvar


###################################################################################################
#Projektbeteiligte
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Projektbeteiligte'):
        found_Cell=True
        og_cell=current_cell
    #print(current_cell)
    #print(found_Cell)
    #print(block.text)
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell
D_Steckbrief['Projektbeteiligte']=textvar

###################################################################################################
#Projektumfeld
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Projektumfeld:'):
        found_Cell=True
        og_cell=current_cell
    #print(current_cell)
    #print(found_Cell)
    #print(block.text)
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell
D_Steckbrief['Projektumfeld']=textvar

###################################################################################################
#Zwischentermine
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Zwischentermine'):
        found_Cell=True
        og_cell=current_cell
    # print(current_cell)
    # print(found_Cell)
    # print(block.text)
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell
D_Steckbrief['Zwischentermine']=textvar

###################################################################################################
#vrsl. Behinderungen/Risiken/Störungen
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('voraussichtliche Behind'):
        found_Cell=True
        og_cell=current_cell
    # print(current_cell)
    # print(found_Cell)
    # print(block.text)
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell
D_Steckbrief['vrsl. Behinderungen/Risiken/Störungen']=textvar

###################################################################################################
#Kunde
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Kunde:'):
        found_Cell=True
        og_cell=current_cell
    # print(current_cell)
    # print(found_Cell)
    # print(block.text)
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell
D_Steckbrief['Kunde']=textvar

###################################################################################################
#Abnahmekriterien
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Abnahmekriterien'):
        found_Cell=True
        og_cell=current_cell
    # print(current_cell)
    # print(found_Cell)
    # print(block.text)
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell
D_Steckbrief['Abnahmekriterien']=textvar


print(D_Steckbrief['Projektbezeichnung'])
print(D_Steckbrief['Projektnummer'])
print(D_Steckbrief['Auftraggeber'])
print(D_Steckbrief['Projektleitung'])
print(D_Steckbrief['Projektorganisationsform'])
print(D_Steckbrief['Oberziel'])
print(D_Steckbrief['Projektinhalt'])
print(D_Steckbrief['Projektbeteiligte'])
print(D_Steckbrief['Projektumfeld'])
print(D_Steckbrief['Zwischentermine'])
print(D_Steckbrief['vrsl. Behinderungen/Risiken/Störungen'])
print(D_Steckbrief['Kunde'])
print(D_Steckbrief['Abnahmekriterien'])


Servicemigration IT-Servicecenter

SC_22

Stephan Bonka, Geschäftsführer (Auftraggeber)

JT



Einführung standardisierter Service- und Betriebsprozesse inkl. Übernahme der Betriebsverantwortung

Initialisierung: Projektstart vorbereiten, Migrationsvertrag liegt vor 
Ist-Aufnahme: Ist Aufnahme der Bank, Rechtevergabe, Vorstellung Projektvorgehen (Workshops)
Analyse: Analyse & Definition der notwendigen Anpassungen
Modellieren: Umsetzung der Anpassungen
Migration: Einrichtung User, Vorbereitung & Schulung der Systeme 
Test: Testfälle durchführen, Betriebsvertrag liegt unterschrieben vor

Auftraggeber (Geschäftsführer): Stephan Bonka
Projektleiterin: JT
Projektteam

Andere IT- Dienstleister für Banken
BaFin (regulatorisch) 

Quality Gate 1 – Dokumentation: 07.11.22
Quality Gate 2 – Gap-Analyse: 26.12.22
Quality Gate 3 – Prozesse: 02.03.23
Quality Gate 4 – Technisch Bereit: 27.04.23
Quality Gate 5 – Betriebsbereit: 29.07.23


Beckedorfer Bank

Vollständige Prozessdokumentation
Betriebsver

NOT ACCURATE DESCRIPTION BUT IT WORKS

Looks for row starting with 'Projektleitung', then looks for cells with 'Leistung', 'Kosten' and 'Termine'  and if it contains '(X)' or '(x)', the dictionary entry is saved as True, if not then as False. 

Could be robuster

In [154]:
#D_Steckbrief['Leistung']=None       #comment out when done testing
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith(D_Steckbrief['Projektleitung'].strip()):
        found_Row=True 
    if found_Row and current_block.text.startswith('Leistung'):
        found_Cell=True
        og_cell=current_cell
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    #print(current_block.text)
    # print(current_block.text.startswith('Leistung'))
    # print(past_blockt.startswith(D_Steckbrief['Projektleitung']))
    #print(found_Cell)
    past_block=current_block
    past_cell=current_cell

#print(textvar)
if '(x)' in textvar.replace(" ", "").lower():
    D_Steckbrief['Leistung']=True
elif '(' in textvar and ')' in textvar:
    D_Steckbrief['Leistung']=False
print('Leistung')
print(D_Steckbrief['Leistung'])

################################################################################################
#Termine 

clear_search_variables()
found_Row=False
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith(D_Steckbrief['Projektleitung'].strip()):
        found_Row=True 
    if found_Row and current_block.text.startswith('Termine'):
        found_Cell=True
        og_cell=current_cell
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    #print(current_block.text)
    # print(current_block.text.startswith('Leistung'))
    # print(past_blockt.startswith(D_Steckbrief['Projektleitung']))
    #print(found_Cell)
    past_block=current_block
    past_cell=current_cell

#print(textvar)
if '(x)' in textvar.replace(" ", "").lower():
    D_Steckbrief['Termine']=True
elif '(' in textvar and ')' in textvar:
    D_Steckbrief['Termine']=False
print('Termine')
print(D_Steckbrief['Termine'])

################################################################################################
#Kosten 

clear_search_variables()
found_Row=False
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith(D_Steckbrief['Projektleitung'].strip()):
        found_Row=True 
    if found_Row and current_block.text.startswith('Kosten'):
        found_Cell=True
        og_cell=current_cell
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    #print(current_block.text)
    # print(current_block.text.startswith('Leistung'))
    # print(past_blockt.startswith(D_Steckbrief['Projektleitung']))
    #print(found_Cell)
    past_block=current_block
    past_cell=current_cell

#print(textvar)
if '(x)' in textvar.replace(" ", "").lower():
    D_Steckbrief['Kosten']=True
elif '(' in textvar and ')' in textvar:
    D_Steckbrief['Kosten']=False
print('Kosten')
print(D_Steckbrief['Kosten'])

Leistung
True
Termine
True
Kosten
False


Looks for row starting with 'Projektorganisationsform', then looks for cells with 'Stabs-PO', 'Matrix-PO' and 'Reine PO'  and if it contains '(X)' or '(x)', the dictionary entry is saved as True, if not then as False. 

Could be robuster

In [129]:
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Projektorganisationsform'):
        found_Row=True 
    if found_Row and current_block.text.startswith('Stabs-PO'):
        found_Cell=True
        og_cell=current_cell
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    #print(current_block.text)
    # print(current_block.text.startswith('Leistung'))
    # print(past_blockt.startswith(D_Steckbrief['Projektleitung']))
    #print(found_Cell)
    past_block=current_block
    past_cell=current_cell

#print(textvar)
if '(x)' in textvar.replace(" ", "").lower():
    D_Steckbrief['Stabs-PO']=True
elif '(' in textvar and ')' in textvar:
    D_Steckbrief['Stabs-PO']=False
print('Stabs-PO')
print(D_Steckbrief['Stabs-PO'])

################################################################################################
#Matrix-PO 

clear_search_variables()
found_Row=False
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Projektorganisationsform'):
        found_Row=True 
    if found_Row and current_block.text.startswith('Matrix-PO'):
        found_Cell=True
        og_cell=current_cell
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    #print(current_block.text)
    # print(current_block.text.startswith('Leistung'))
    # print(past_blockt.startswith(D_Steckbrief['Projektleitung']))
    #print(found_Cell)
    past_block=current_block
    past_cell=current_cell

#print(textvar)
if '(x)' in textvar.replace(" ", "").lower():
    D_Steckbrief['Matrix-PO']=True
elif '(' in textvar and ')' in textvar:
    D_Steckbrief['Matrix-PO']=False
print('Matrix-PO')
print(D_Steckbrief['Matrix-PO'])

################################################################################################
#Reine PO 

clear_search_variables()
found_Row=False
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Projektorganisationsform'):
        found_Row=True 
    if found_Row and current_block.text.startswith('Reine PO'):
        found_Cell=True
        og_cell=current_cell
    if found_Cell:
        if current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    #print(current_block.text)
    # print(current_block.text.startswith('Leistung'))
    # print(past_blockt.startswith(D_Steckbrief['Projektleitung']))
    #print(found_Cell)
    past_block=current_block
    past_cell=current_cell

#print(textvar)
if '(x)' in textvar.replace(" ", "").lower():
    D_Steckbrief['Reine PO']=True
elif '(' in textvar and ')' in textvar:
    D_Steckbrief['Reine PO']=False
print('Reine PO')
print(D_Steckbrief['Reine PO'])

Stabs-PO
False
Matrix-PO
True
Reine PO
False


Extracts Geplante Termine (wann?):
* Start
* Ende
*  Dauer

In [31]:
#Start
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Geplante Termine') and current_block.text.startswith('Start:'):
        found_Cell=True
        og_cell=current_cell
    if found_Cell:
        if current_block.text.startswith('Ende:') or current_cell!=og_cell: 
            break
        textvar+=block.text #took this line into the if condition
        textvar+="\n"
    past_block=current_block
    past_cell=current_cell

textvar=textvar.replace('Start:','')
textvar=textvar.lstrip() #removes tab
textvar=textvar.strip() #removes eventual whitespaces in the begining or end
D_Steckbrief['Starttermin']=textvar

###################################################################################################
#Ende
clear_search_variables()
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr,cellnr)
    if past_block!='' and past_block.text.startswith('Geplante Termine') and current_block.text.startswith('Start:'):
        found_Cell=True
        og_cell=current_cell
    if found_Cell:
        if current_block.text.startswith('Ende:'):
            if current_cell!=og_cell: 
                break
            textvar+=block.text #took this line into the if condition
            textvar+="\n"
    past_block=current_block
    past_cell=current_cell
#print(textvar)
textvar=textvar.replace('Ende:','')
textvar=textvar.lstrip() #removes tab
textvar=textvar.strip() #removes eventual whitespaces in the begining or end
D_Steckbrief['Endetermin']=textvar


###################################################################################################
#Abnahmekriterien
clear_search_variables()
found_Row=False
for block in iter_block_items_n(doc):
    current_block=block
    current_Row=rownr
    current_cell=(rownr,cellnr)
    if past_block!='' and current_block.text.startswith('Geplante Termine'):
        found_Row=True
        og_Row=current_Row
    if found_Row:
        if current_Row!=og_Row: 
            break
        if current_block.text.startswith('Dauer:'):
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell: 
                break
            textvar+=block.text #took this line into the if condition
            textvar+="\n"
            break
    past_block=current_block
    past_cell=current_cell
#print(textvar)
textvar=textvar.replace('Dauer:','')
textvar=textvar.lstrip() #removes tab
textvar=textvar.strip() #removes eventual whitespaces in the begining or end
D_Steckbrief['Dauer']=textvar
# print(textvar)

del found_Row

print(D_Steckbrief['Starttermin'])
print(D_Steckbrief['Endetermin'])
print(D_Steckbrief['Dauer'])


01.09.22
01.09.23
12 Monate


Function definition to look for all rows with gesamt, intern, extern cells.

* Looks for cell after "Aufwand [PT]" and saves "gesamt" into dictionary entry (Start logic), remembers where gesamt is using flag_cell

* Looks for cell after "gesamt" using flag cell and saves "intern" as dictionary entry (maybe should be more robust), updates flag_cell

* Looks for cell after "intern" usign flag_cell and saves "extern" as dictionary entry (Same logic like code block for "intern", maybe should be more robust)

In [46]:
def G_I_E_CellSearch(row_heading,keys_list):
    global past_block, found_Cell, past_Cell, textvar
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_block=block
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith(row_heading) and current_block.text.startswith('gesamt'):
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_block.text.startswith('intern:') or current_cell!=og_cell: 
                break
            textvar+=block.text #took this line into the if condition
            textvar+="\n"
        past_block=current_block
        past_cell=current_cell

    textvar=textvar.replace('gesamt','')
    textvar=textvar.lstrip() #removes tab
    textvar=textvar.strip() #removes eventual whitespaces in the begining or end
    D_Steckbrief[keys_list[0]]=textvar
    #print(textvar)
    print(D_Steckbrief[keys_list[0]])
    flag_cell=(current_cell[0],current_cell[1]-1) #this is the cell number of the cell with "gesamt" we just found, it is not a combined cell and therefore doesn't repeat itself
    #print(flag_cell)
    # ***********************************************************************************
    # NEW CELL STARTS

    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_block=block
        current_cell=(rownr,cellnr)
        #print(current_cell)
        #print(current_block.text)
        if past_block!='' and past_block.text.startswith('gesamt') and current_block.text.startswith('intern') and past_Cell==flag_cell:
            found_Cell=True
            #print('found intern')
            og_cell=current_cell
        if found_Cell:
            if current_block.text.startswith('extern:') or current_cell!=og_cell: 
                break
            textvar+=block.text #took this line into the if condition
            textvar+="\n"
        past_block=current_block
        past_Cell=current_cell

    textvar=textvar.replace('intern','')
    textvar=textvar.lstrip() #removes tab
    textvar=textvar.strip() #removes eventual whitespaces in the begining or end
    D_Steckbrief[keys_list[1]]=textvar
    #print(textvar)
    print(D_Steckbrief[keys_list[1]])
    flag_cell=(current_cell[0],current_cell[1]+2) #this is the cell number of the last repeated cell "intern" (it repeats because it is a combined cell)
    #print(flag_cell)

    # ***********************************************************************************
    # NEW CELL STARTS

    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_block=block
        current_cell=(rownr,cellnr)
        #print(current_cell)
        #print(current_block.text)
        if past_block!='' and past_block.text.startswith('intern') and current_block.text.startswith('extern') and past_Cell==flag_cell:
            found_Cell=True
            #print('found extern')
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell: 
                break
            textvar+=block.text #took this line into the if condition
            textvar+="\n"
        past_block=current_block
        past_Cell=current_cell

    textvar=textvar.replace('extern','')
    textvar=textvar.lstrip() #removes tab
    textvar=textvar.strip() #removes eventual whitespaces in the begining or end
    D_Steckbrief[keys_list[2]]=textvar
    #print(textvar)
    print(D_Steckbrief[keys_list[2]])

In [49]:
row_heading='Aufwand [PT]:'
keys_list_Aufwand=['Aufwand-gesamt','Aufwand-intern','Aufwand-extern']
print('Aufwand')
G_I_E_CellSearch(row_heading,keys_list_Aufwand)

row_heading='Personalkosten ['
keys_list_Aufwand=['Personalkosten-gesamt','Personalkosten-intern','Personalkosten-extern']
print('Personalkosten')
G_I_E_CellSearch(row_heading,keys_list_Aufwand)

row_heading='Sachkosten ['
keys_list_Aufwand=['Sachkosten-gesamt','Sachkosten-intern','Sachkosten-extern']
print('Sachkosten')
G_I_E_CellSearch(row_heading,keys_list_Aufwand)

row_heading='Investitionen ['
keys_list_Aufwand=['Investitionen-gesamt','Investitionen-intern','Investitionen-extern']
print('Investitionen')
G_I_E_CellSearch(row_heading,keys_list_Aufwand)

row_heading='Budget ['
keys_list_Aufwand=['Budget-gesamt','Budget-intern','Budget-extern']
print('Budget')
G_I_E_CellSearch(row_heading,keys_list_Aufwand)


Aufwand
214
214
0
Personalkosten
120.485,02
120.485,02
0
Sachkosten
0
0
0
Investitionen
0
0
0
Budget
120.485,02
120.485,02
0


TO DO: JUST COPIED, NOT YET ADJUSTED TO ZIELE

3.2 Ziele

* Extracts Oberziel
* Extracts infos from the Ziele table (Using Änderungshistorie logic)
* Extracts infos from the Nicht-Ziele table

In [150]:
clear_search_variables()
left_Table=False
flag_cell=(2,2)
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr, cellnr)
    if past_block!='' and past_block.text.startswith('Nr.') and current_block.text.startswith('Hierarchie'):
        found_Table=True
    if found_Table and current_cell==(0,0):
        left_Table=True
        break
    if found_Table and past_Cell==flag_cell and current_cell[1]>flag_cell[1] and past_block.text.startswith('Oberziel'):
        found_Cell=True
        og_cell=current_cell
    if found_Cell:
        if current_cell!=og_cell:
            break
        textvar+=block.text
        textvar+='\n'
    past_Cell=current_cell
    past_block=block
flag_cell=og_cell
#print('flag cell after Oberziel:')
#print(flag_cell)
D_Oberziel['Bezeichnung']=textvar


################################################################################
#Beschreibung

clear_search_variables()
left_Table=False
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr, cellnr)
    if past_block!='' and past_block.text.startswith('Nr.') and current_block.text.startswith('Hierarchie'):
        found_Table=True
    if found_Table and current_cell==(0,0):
        left_Table=True
        break
    if found_Table and past_Cell==flag_cell and current_cell[1]>flag_cell[1] and past_block.text.startswith(D_Oberziel['Bezeichnung'].strip()):
        found_Cell=True
        og_cell=current_cell
    if found_Cell:
        if current_cell!=og_cell:
            break
        textvar+=block.text
        textvar+='\n'
    #print(current_cell)
    #print(current_block.text)
    #print(found_Cell)
    past_Cell=current_cell
    past_block=block
flag_cell=og_cell
D_Oberziel['Formulierung']=textvar

################################################################################
#Messkriterium

clear_search_variables()
left_Table=False
for block in iter_block_items_n(doc):
    current_block=block
    current_cell=(rownr, cellnr)
    if past_block!='' and past_block.text.startswith('Nr.') and current_block.text.startswith('Hierarchie'):
        found_Table=True
    if found_Table and current_cell==(0,0):
        left_Table=True
        break
    if found_Table and past_Cell==flag_cell and current_cell[1]>flag_cell[1] and past_block.text.startswith(D_Oberziel['Formulierung'].strip()):
        found_Cell=True
        og_cell=current_cell
    if found_Cell:
        if current_cell!=og_cell:
            break
        textvar+=block.text
        textvar+='\n'
    past_Cell=current_cell
    past_block=block
flag_cell=og_cell
D_Oberziel['Messkriterium']=textvar

print(D_Oberziel['Bezeichnung'])
print(D_Oberziel['Formulierung'])
print(D_Oberziel['Messkriterium'])


Einführung standardisierter Service- u. Betriebsprozesse inkl. Übernahme IT-Betriebs-verantwortung 

Einführung standardisierter Service- u. Betriebsprozesse werden Aufgaben u. Prozesse der IT-Abteilung der Bank an die ABC GmbH teils übertragen, neu strukturiert u. gegebenenfalls erweitert

Vollständige Prozessdokumentation. Betriebsvertrag muss vorliegen u. die Unterschriebene Abnahmeerklärung liegt vor. Standardisierte Service- &. Betriebsprozesse sind eingeführt und die IT-Betriebsverantwortung wurde von der ABC GmbH übernommen.




In [ ]:
clear_search_variables()
left_Table=False
flag_cell=(2,2)
while not (found_Table and current_cell==(0,0)):
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Nr.') and current_cell==(1,2) and current_block.text.startswith('Hierarchie'):
            found_Table=True
        if found_Table and current_cell==(0,0):
            left_Table=True
            break
        if found_Table and past_Cell==flag_cell and current_cell[0]>flag_cell[0] and past_block.text.startswith('Oberziel'):
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
        past_Cell=current_cell
        past_block=block
        
    version_nr=textvar
    new_D_key='Version_'+version_nr
    flag_cell=og_cell
    D_Änderungshistorie[new_D_key]={}
    #print(D_Änderungshistorie)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    # print('flag_cell: ', end='')
    # print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Änderungshistorie') and current_cell==(1,1):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block
    version_datum=textvar
    new_D_value=version_datum
    flag_cell=og_cell
    D_Änderungshistorie[new_D_key]['Datum']=new_D_value
    #print(D_Änderungshistorie)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Änderungshistorie') and current_cell==(1,1):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
        past_Cell=current_cell
        past_block=block
    version_ersteller=textvar
    new_D_value=version_ersteller
    flag_cell=og_cell
    D_Änderungshistorie[new_D_key]['Ersteller']=new_D_value
    #print(D_Änderungshistorie)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Änderungshistorie') and current_cell==(1,1):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
        past_Cell=current_cell
        past_block=block
    
    version_grund=textvar
    new_D_value=version_grund
    flag_cell=og_cell
    D_Änderungshistorie[new_D_key]['Grund']=new_D_value
    #print(D_Änderungshistorie)
print(D_Änderungshistorie)

Nichtziele

In [38]:
clear_search_variables()
D_NichtZiele={} #Further entries are added later on 
left_Table=False
flag_cell=(2,2)
i=0
while not (found_Table and current_cell==(0,0)):
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Nicht-Ziele (out of scope)') and current_cell==(1,2): #the block 'Nicht-Ziele (out of scope)' repeats itself because it is in a merged table cell
            found_Table=True
        if found_Table and current_cell==(0,0):
            left_Table=True
            break
        if found_Table and past_Cell==flag_cell and current_cell[0]>flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        past_Cell=current_cell
        past_block=block

    i+=1    
    nichtziel_nr=str(i)
    new_D_key='NZ'+nichtziel_nr
    new_D_value=textvar.strip()
    flag_cell=og_cell
    D_NichtZiele[new_D_key]={}
    D_NichtZiele[new_D_key]['Zielbezeichnung']=new_D_value
    #print(D_NichtZiele)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    # print('flag_cell: ', end='')
    # print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Nicht-Ziele (out of scope)') and current_cell==(1,2):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block
    new_D_value=textvar.strip()
    flag_cell=og_cell
    D_NichtZiele[new_D_key]['Zielbeschreibung']=new_D_value
    #print(D_NichtZiele)

del i

print(D_NichtZiele)


{'NZ1': {'Zielbezeichnung': 'Datenschutzverordnung', 'Zielbeschreibung': 'Keine Verletzung des Datenschutzes. Wahrung der Integrität und Wahrung der Kundendaten'}, 'NZ2': {'Zielbezeichnung': 'Beeinträchtigung Bankbetrieb', 'Zielbeschreibung': 'Keine Beeinträchtigung des laufenden Bankbetriebs während der Migration.'}}


TO DO: CHECAR CON OTRO EJEMPLO

3.3 Extracts infos from the Zielkonflikte table

In [39]:
clear_search_variables()
D_Zielkonflikte={} #Further entries are added later on 
left_Table=False
flag_cell=(1,6)
i=0
while not (found_Table and current_cell==(0,0)):
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if  block.text.startswith('Zielkonflikt') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Zielkonflikt'):
            found_Table=True
        if found_Table and current_cell==(0,0):
            left_Table=True
            break
        if found_Table and  current_cell[0]>flag_cell[0] and current_cell[1]==1:    #and past_Cell==flag_cell with the JT example it doesn't work because he added an extra column
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        print(current_cell)
        print(block.text)
        print(found_Cell)
        past_Cell=current_cell
        past_block=block

    i+=1    
    Zielkonflikt_nr=str(i)
    new_D_key='ZK'+Zielkonflikt_nr
    new_D_value=textvar.strip()
    flag_cell=og_cell
    D_Zielkonflikte[new_D_key]={}
    D_Zielkonflikte[new_D_key]['Ziel 1']=new_D_value
    #print(D_Zielkonflikte)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    # print('flag_cell: ', end='')
    # print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if  block.text.startswith('Zielkonflikt') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Zielkonflikt'):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block
    new_D_value=textvar.strip()
    flag_cell=og_cell
    D_Zielkonflikte[new_D_key]['Ziel 2']=new_D_value
    #print(D_Zielkonflikte)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    # print('flag_cell: ', end='')
    # print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if  block.text.startswith('Zielkonflikt') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Zielkonflikt'):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_block.text!=block.text and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block
    new_D_value=textvar.strip()
    flag_cell=og_cell
    D_Zielkonflikte[new_D_key]['Art']=new_D_value
    #print(D_Zielkonflikte)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    # print('flag_cell: ', end='')
    # print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if  block.text.startswith('Zielkonflikt') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Zielkonflikt'):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block
    new_D_value=textvar.strip()
    flag_cell=og_cell
    D_Zielkonflikte[new_D_key]['Priorität']=new_D_value
    #print(D_Zielkonflikte)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    # print('flag_cell: ', end='')
    # print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if  block.text.startswith('Zielkonflikt') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Zielkonflikt'):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block
    new_D_value=textvar.strip()
    flag_cell=og_cell
    D_Zielkonflikte[new_D_key]['Erklärung']=new_D_value
    #print(D_Zielkonflikte)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    # print('flag_cell: ', end='')
    # print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if  block.text.startswith('Zielkonflikt') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Zielkonflikt'):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block
    new_D_value=textvar.strip()
    flag_cell=og_cell
    D_Zielkonflikte[new_D_key]['Massnahmen']=new_D_value
    #print(D_Zielkonflikte)

del i

print(D_Zielkonflikte)


(0, 0)

False
(0, 0)

False
(0, 0)

False
(0, 0)

False
(0, 0)

False
(0, 0)

False
(0, 0)
Report Level D
False
(0, 0)

False
(0, 0)
nach Z01D_Leitfaden / 02 vom 20.06.2023
False
(0, 0)

False
(0, 0)
Dieses Dokument basiert auf der Report-Vorlage der Decisio Projekt- und Prozessmanagement GmbH. 
False
(0, 0)

False
(0, 0)

False
(0, 0)

False
(1, 1)
Projekt:
False
(1, 2)
Servicemigration IT-Servicecenter
False
(2, 1)
Teilnehmer:
False
(2, 2)
JT
False
(3, 1)
Firma:
False
(3, 2)
ABC
False
(4, 1)
Adresse:
False
(4, 2)

False
(5, 1)
Telefon:
False
(5, 2)

False
(6, 1)
E-Mail:
False
(6, 2)

False
(7, 1)
Kurs:
False
(7, 2)
23-1111
False
(0, 0)

False
(0, 0)

False
(0, 0)

False
(0, 0)
Änderungshistorie	
False
(1, 1)
Version
False
(1, 2)
Datum
False
(1, 3)
Ersteller
False
(1, 4)
Grund
False
(2, 1)
0.1
False
(2, 2)
26.09.23
False
(2, 3)
JT
False
(2, 4)
Ersterstellung
False
(3, 1)
0.2
False
(3, 2)
26.11.23
False
(3, 3)
JT
False
(3, 4)
Überarbeitung
False
(4, 1)
1.0
False
(4, 2)
tt.mm.jj
False
(

## Chapter 4: Stakeholder 4.5.12.

4.1 Extracts infos from the Projektumfeld table

In [40]:
#del so_in_var, so_ex_var, sa_in_var, sa_ex_var #use for making changes in this code
clear_search_variables()
#D_Projektumfeld={'sozial-intern':'',
#                 'sozial-extern':'',
#                 'sachlich-intern':'',
#                 'sachlich-extern':''}
left_Table=False
flag_cell=(2,1)#cell containing the keyword we're looking for. In this case the cell containing 'sozial'
while not (found_Table and current_cell==(0,0)):
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Projektumfeld') and past_block.style.name=='Heading 2' and current_cell==(1,1):
            found_Table=True
        if found_Table and current_cell==(0,0):
            left_Table=True
            break
        if found_Table and past_Cell==flag_cell and past_block.text.startswith('sozial') and current_cell[1]>flag_cell[1]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
#            print(current_cell)
#            print('textvar')
#            print(textvar)
            textvar+=block.text
            textvar+="\n"
#        if found_Cell:
#            print(current_cell)
#            print(block.text)
#            print(current_cell==og_cell)
        past_Cell=current_cell
        past_block=block
    so_in_var=textvar.strip()
    flag_cell=og_cell
    D_Projektumfeld['sozial-intern']=so_in_var
    #print(D_Projektumfeld)

    
    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################


    #print('flag_cell: ', end='')
    #print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Projektumfeld') and past_block.style.name=='Heading 2' and current_cell==(1,1):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block
    so_ex_var=textvar.strip()
    flag_cell=og_cell
    D_Projektumfeld['sozial-extern']=so_ex_var
    #print(D_Projektumfeld)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    flag_cell=(3,1)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Projektumfeld') and past_block.style.name=='Heading 2' and current_cell==(1,1):
            found_Table=True
        if found_Table and current_cell==(0,0):
            left_Table=True
            break
        if found_Table and past_Cell==flag_cell and past_block.text.startswith('sachlich') and current_cell[1]>flag_cell[1]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
#            print(current_cell)
#            print('textvar')
#            print(textvar)
            textvar+=block.text
            textvar+="\n"
#        if found_Cell:
#            print(current_cell)
#            print(block.text)
#            print(current_cell==og_cell)
        past_Cell=current_cell
        past_block=block
    sa_in_var=textvar.strip()
    flag_cell=og_cell
    D_Projektumfeld['sachlich-intern']=sa_in_var
    #print(D_Projektumfeld)


    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################


    #print('flag_cell: ', end='')
    #print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Projektumfeld') and past_block.style.name=='Heading 2' and current_cell==(1,1):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block
    sa_ex_var=textvar.strip()
    flag_cell=og_cell
    D_Projektumfeld['sachlich-extern']=sa_ex_var
print(D_Projektumfeld)


{'sozial-intern': 'S1: Projektteam (PO: Julia Hartel; MA in Linie: Thorsten Pitha, Judith Neumer, Lara Uckun, Nora Müller, Nico Bausch, Peer Arnemann, Marvin Lose, Christel Bauer)\nS2: Geschäftsführer: Volker Möller\nS3: PL JT', 'sozial-extern': 'S4: Auftraggeber (GF): \nStephan Bonka\nS4: Projektleiterin & Abteilungsleiterin IT-Betrieb Bank: \nUrsula Bittger\nS7: MA Bank AP 5 & 6: \nTorben Groß', 'sachlich-intern': 'R1: Projektergebnisse\nR2: Verletzung Datenschutzgesetz\nR3: Vorlage Betriebsvertrag', 'sachlich-extern': 'R4: Einhaltung Regulatorik (MaRisk)\nR5: Prozessdokumentation\nR6: Überarbeitung Netzwerkschränke'}


4.2 Extracts infos from the Schnittstellen Stakeholder - Umfeld table

In [41]:
clear_search_variables()
D_Schnittstellen_Stakeholder_Umfeld={} #Further entries are added later on 
left_Table=False
flag_cell=(1,3)
i=0
while not (found_Table and current_cell==(0,0)):
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Schnittstellen Stakeholder - Umfeld') and current_cell==(1,1):
            found_Table=True
        if found_Table and current_cell==(0,0):
            left_Table=True
            break
        if found_Table and past_Cell==flag_cell and current_cell[0]>flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        past_Cell=current_cell
        past_block=block
        
    i+=1    
    SSU_nr=str(i)
    new_D_key='SSU'+SSU_nr
    new_D_value=textvar.strip()
    flag_cell=og_cell
    D_Schnittstellen_Stakeholder_Umfeld[new_D_key]={}
    D_Schnittstellen_Stakeholder_Umfeld[new_D_key]['Schnittstelle']=new_D_value
    #print(D_Schnittstellen_Stakeholder_Umfeld)
    #print(i)
    #print(SSU_nr)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    # print('flag_cell: ', end='')
    # print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Schnittstellen Stakeholder - Umfeld') and current_cell==(1,1):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block

    
    SSU_Ansprechpartner=textvar.strip()
    new_D_value=SSU_Ansprechpartner
    flag_cell=og_cell
    D_Schnittstellen_Stakeholder_Umfeld[new_D_key]['Ansprechpartner']=new_D_value
    #print(D_Schnittstellen_Stakeholder_Umfeld)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and past_block.text.startswith('Schnittstellen Stakeholder - Umfeld') and current_cell==(1,1):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        past_Cell=current_cell
        past_block=block
    SSU_Auswirkung=textvar.strip()
    new_D_value=SSU_Auswirkung
    flag_cell=og_cell
    D_Schnittstellen_Stakeholder_Umfeld[new_D_key]['Auswirkung']=new_D_value
    #print(D_Schnittstellen_Stakeholder_Umfeld)

del i, new_D_key, new_D_value
print(D_Schnittstellen_Stakeholder_Umfeld)

{'SSU1': {'Schnittstelle': 'Projektergebnisse: Die Dokumentation der Ergebnisse ist erforderlich und muss in die Linie übergeben werden.', 'Ansprechpartner': 'Thorsten Pitha', 'Auswirkung': 'Zur Sicherstellung des Know-How Transfers von Projektmitarbeitern auf MA in der Linie muss eine Übergabe erfolgen. Für die Umsetzung späterer Betriebsprojekte ggfs. relevant.'}, 'SSU2': {'Schnittstelle': 'Vorlage Betriebsvertrag: Der Betriebsübergang darf offiziell nur mit unterschriebenen Betriebsvertrag erfolgen', 'Ansprechpartner': 'Ursula Bittger', 'Auswirkung': 'Falls der Betriebsvertrag nicht vorliegt, muss der Betriebsübergang verschoben werden und das Budget des Projektes kann nicht eingehalten werden. Aus Sicht des GF ist eine Verschiebung nicht erwünscht.'}}


4.3 Ignored, we don't consider images

4.4 Extracts infos from the Stakeholderinteressen table

In [42]:
clear_search_variables()
D_Stakeholderinteressen={} #Further entries are added later on 
left_Table=False
flag_cell=(1,7)
while not (found_Table and current_cell==(0,0)):
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if  block.text.startswith('Stakeholderinteressen') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Stakeholder'):
            found_Table=True
        if found_Table and current_cell==(0,0):
            left_Table=True
            break
        if found_Table and past_Cell==flag_cell and current_cell[0]>flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        #print(current_cell)
        #print(block.text)
        #print(found_Table)
        past_Cell=current_cell
        past_block=block

    if textvar:
        new_D_key=textvar.strip()
        flag_cell=og_cell
        D_Stakeholderinteressen[new_D_key]={}
    #print(D_Stakeholderinteressen)
    #print(i)
    #print(SSU_nr)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    # print('flag_cell: ', end='')
    # print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and block.text.startswith('Stakeholderinteressen') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Stakeholder'):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block

    if textvar:
        S_Konfliktpotentzial=textvar.strip()
        new_D_value=S_Konfliktpotentzial
        flag_cell=og_cell
        D_Stakeholderinteressen[new_D_key]['Konfliktpotenzial']=new_D_value
    #print(D_Stakeholderinteressen)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and block.text.startswith('Stakeholderinteressen') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Stakeholder'):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        past_Cell=current_cell
        past_block=block
    if textvar:    
        S_Einfluss=textvar.strip()
        new_D_value=S_Einfluss
        flag_cell=og_cell
        D_Stakeholderinteressen[new_D_key]['Einfluss']=new_D_value
    #print(D_Stakeholderinteressen)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    # print('flag_cell: ', end='')
    # print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and block.text.startswith('Stakeholderinteressen') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Stakeholder'):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block

    if textvar:
        S_Interessen_SH_Projekt=textvar.strip()
        new_D_value=S_Interessen_SH_Projekt
        flag_cell=og_cell
        D_Stakeholderinteressen[new_D_key]['Interessen SH-Projekt']=new_D_value
    #print(D_Stakeholderinteressen)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and block.text.startswith('Stakeholderinteressen') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Stakeholder'):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        past_Cell=current_cell
        past_block=block
    if textvar:    
        S_Interessen_Projekt_SH=textvar.strip()
        new_D_value=S_Interessen_Projekt_SH
        flag_cell=og_cell
        D_Stakeholderinteressen[new_D_key]['Interessen Projekt-SH']=new_D_value
    #print(D_Stakeholderinteressen)


    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    # print('flag_cell: ', end='')
    # print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and block.text.startswith('Stakeholderinteressen') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Stakeholder'):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block

    if textvar:
        S_Strategie=textvar.strip()
        new_D_value=S_Strategie
        flag_cell=og_cell
        D_Stakeholderinteressen[new_D_key]['Strategie']=new_D_value
    #print(D_Stakeholderinteressen)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and block.text.startswith('Stakeholderinteressen') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Stakeholder'):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
            textvar+="\n"
        past_Cell=current_cell
        past_block=block
    if textvar:    
        S_Steuerung=textvar.strip()
        new_D_value=S_Steuerung
        flag_cell=og_cell
        D_Stakeholderinteressen[new_D_key]['Steuerung Massnahmen']=new_D_value
    #print(D_Stakeholderinteressen)



print(D_Stakeholderinteressen)

{'T. Pitha\nMA Linie (FP)': {'Konfliktpotenzial': 'niedrig', 'Einfluss': 'Niedrig', 'Interessen SH-Projekt': '- Anerkennung für die Mitwirkung\n- Zu wenig Know-how bei den Mitarbeitern der Bank', 'Interessen Projekt-SH': '- Sorgt durch seine Fachkenntnisse für einen reibunglosen Ablauf im Projekt\n- fehlende proaktive Kommunikation bei Engpässen', 'Strategie': 'informativ', 'Steuerung Massnahmen': '1x wöchentlich interner Jour Fixé im Projektteam zum Status, Abarbeiten der Punkte des Projektplans & pflege dieser, führen einer gemeinsamen OPL + Erledigung der Punkte;\n1x wöchentlich externer AP JF Team der Bank'}, 'S. Bonka\nAG (GF) (MP)': {'Konfliktpotenzial': 'niedrig', 'Einfluss': 'Hoch', 'Interessen SH-Projekt': '- schnelle und effiziente Umsetzung ohne große Probleme, Minimierung der Personalrisiken\n- Einschränkungen im Tagesgeschäft durch das Projekt', 'Interessen Projekt-SH': '- Unterstützung des Projektes hinsichtlich evtl. Krisenbewältigung; Setzt sich bei seinen MA für das Pr

## Chapter 5: Macht und Interessen 04.03.04 

TO DO: ADD A SPACE AFTER EVERY BLOCK INSIDE A CELL

5.1 Extracts infos from the Stakeholderbewertung table 

In [145]:
clear_search_variables()
D_Stakeholderbewertung={} #Further entries are added later on 
left_Table=False
flag_cell=(1,3)
while not (found_Table and current_cell==(0,0)):
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if  block.text.startswith('Stakeholderbewertung') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Stakeholder'):
            found_Table=True
        if found_Table and current_cell==(0,0):
            left_Table=True
            break
        if found_Table and past_Cell==flag_cell and current_cell[0]>flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
        #print(current_cell)
        #print(block.text)
        #print(found_Table)
        past_Cell=current_cell
        past_block=block

    if textvar:
        first_word=textvar.split()[0]

        getVals = list([val for val in first_word if val.isalpha() or val.isnumeric()])
        Stakeholder_nr = "".join(getVals)

        new_D_key=Stakeholder_nr
        new_D_value=textvar.split(None, 1)[1]
        flag_cell=og_cell
        D_Stakeholderbewertung[new_D_key]={}
        D_Stakeholderbewertung[new_D_key]['Name, Funktion']=new_D_value
    #print(D_Stakeholderbewertung)
    #print(i)
    #print(SSU_nr)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    # print('flag_cell: ', end='')
    # print(flag_cell)
    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and block.text.startswith('Stakeholderbewertung') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Stakeholder'):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
            # print('og_cell: ', end='')
            # print(og_cell)
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
        # print(past_Cell, end='    ')
        # print(current_cell[1], end=', ')
        # print(flag_cell[1])
        # print(block.text)
        # #print(current_cell)
        # print('found table: ', end='')
        # print(found_Table, end='    ')
        # print('found cell: ', end='')
        # print(found_Cell)
        past_Cell=current_cell
        past_block=block

    if textvar:
        Sn_Macht=textvar
        new_D_value=Sn_Macht
        flag_cell=og_cell
        D_Stakeholderbewertung[new_D_key]['Macht']=new_D_value
    #print(D_Stakeholderbewertung)

    ######################################################################################################################################
    #NEW CELL
    ######################################################################################################################################

    clear_search_variables()
    for block in iter_block_items_n(doc):
        current_cell=(rownr,cellnr)
        if past_block!='' and block.text.startswith('Stakeholderbewertung') and block.style.name=='Heading 2':
            found_Chapter=True
        if found_Chapter and current_cell==(1,1) and block.text.startswith('Stakeholder'):
            found_Table=True
        if found_Table and past_Cell==flag_cell and past_Cell!=current_cell and current_cell[0]==flag_cell[0]:
            found_Cell=True
            og_cell=current_cell
        if found_Cell:
            if current_cell!=og_cell:
                break
            textvar+=block.text
        past_Cell=current_cell
        past_block=block
    if textvar:    
        Sn_Begründung=textvar
        new_D_value=Sn_Begründung
        flag_cell=og_cell
        D_Stakeholderbewertung[new_D_key]['Begründung der Machtposition']=new_D_value
    #print(D_Stakeholderbewertung)

print(D_Stakeholderbewertung)

{'S1': {'Name, Funktion': 'T. Pitha MA in Linie (AP1) ', 'Macht': 'Niedrig', 'Begründung der Machtposition': 'Verfügt über eine hohe Fachexpertise, ist jedoch normaler Mitarbeiter in der Linie und fachlicher Verantwortlicher für das Arbeitspaket 1.'}, 'S2': {'Name, Funktion': 'V. MöllerGeschäftsführer', 'Macht': 'Hoch', 'Begründung der Machtposition': 'Aufgrund seiner Position als Geschäftsführer der ABC GmbH verfügt er über die Befugnis disziplinarisch und auch fachlich gegenüber seinen Mitarbeitern zu weisen.'}, 'S5': {'Name, Funktion': 'S. BonkaAuftraggeber (Geschäftsführer)', 'Macht': 'Hoch', 'Begründung der Machtposition': 'Aufgrund seiner Position als Geschäftsführer der Beckedorfer Bank hat er die nötige disziplinarische und auch fachliche Weisungsbefugnis gegenüber seinen Mitarbeitern.'}, 'S6': {'Name, Funktion': 'U. BittgerProjekt- & Abteilungsleiterin', 'Macht': 'Hoch', 'Begründung der Machtposition': 'Aufgrund ihrer fachlichen und disziplinarischen Weisungsbefugnis als Abtei

TO DO

5.2 Looks for paragraphs in Subchapter 5.2 'Machtpromotoren' and saves them into a directory

In [69]:
# The text of the chapter will be saved into this variable
clear_search_variables()
for block in iter_block_items(doc):
    current_block=block
    # don't need to ignore repeated text from combined cells for this case
    #Are we in the section we're looking for based on a previous block?
    if past_block!='' and past_block.style.name=='Heading 2' and past_block.text.startswith('Machtpromotoren'):
        found_Chapter=True
    #Break argument. When are we leaving the section we're lookin for?
    if found_Chapter:
        if block.style.name=='Heading 2' or block.style.name=='Heading 1':
            break
        textvar+=block.text
    #print(block.text)
    past_block=current_block
D_Machtpromotoren['Machtpromotoren']=textvar
print(D_Machtpromotoren['Machtpromotoren'])

In diesem Projekt ist der Geschäftsführer der Beckedorfer Bank Herr Stephan Bonka der Macht-promotor.Angesichts der Entwicklungen von Kosten und Personal ist eine ordnungsgemäße Betriebsführung in der Bank zunehmend schwer aufrecht zu erhalten weshalb Risiken steigen. Ausgebildete und hochqualifizierte IT-Spezialisten sind für die Bank schwer zu akquirieren. Aus diesen Gründen ist Herr Bonka Befürworter dieses Projektes und promotet es intern bei seinen Mitarbeitern.Jedoch ist er in seiner Funktion als Geschäftsführer auch dazu in der Lage, dass der Betriebsüber-gang aufgrund von unzufriedenstellenden Mängeln und daraus resultierender Betriebsrisiken ver-schoben wird oder aber eine Abnahme verweigert wird.
